In [1]:
import os
from nbed.driver import NbedDriver
import numpy as np

/home/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# get xyz file for water
notebook_dir = os.getcwd()
NBed_dir = os.path.dirname(notebook_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [3]:
geometry_path = os.path.join(mol_dir, 'water.xyz')

In [4]:
# geometry_path = os.path.join(mol_dir, 'H4.xyz')

In [5]:
with open(geometry_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

3

O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [6]:
from nbed.utils import pubchem_mol_geometry, build_ordered_xyz_string
mol_name = 'NH3'


struct_dict = pubchem_mol_geometry(mol_name)
active_atom_inds= [1,2]
xyz_string = build_ordered_xyz_string(struct_dict, active_atom_inds)
print(xyz_string)

4
 
H	-0.4417	0.2906	0.8711
H	0.7256	0.6896	-0.1907
N	0	0	0
H	0.4875	-0.8701	0.2089



In [7]:
# options
geometry = xyz_string
n_active_atoms=2
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = False
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=500


projector = 'both'
localization = 'ibo' # spade, ibo
occupied_threshold = 0.3

In [ ]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    _init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 2.75e-11


In [ ]:
print(driver.localized_system.active_MO_inds)
print(driver.localized_system.enviro_MO_inds)

## access hidden parmeters

In [ ]:
mu_scf = driver._mu["scf"]
huz_scf = driver._huzinaga["scf"]
global_scf = driver._global_hf

In [ ]:
c_mat = mu_scf.mo_coeff
mo_occ = mu_scf.get_occ()
dmat = mu_scf.make_rdm1(mo_coeff=c_mat, mo_occ=mo_occ)
F =  mu_scf.get_fock(dm=dmat)

mu_CFC = c_mat.T @ F @c_mat
print(np.around(mu_CFC,
         3))

In [ ]:
c_mat = huz_scf.mo_coeff
mo_occ = huz_scf.get_occ()
dmat = huz_scf.make_rdm1(mo_coeff=c_mat, mo_occ=mo_occ)
F =  huz_scf.get_fock(dm=dmat)

huz_CFC = c_mat.T @ F @c_mat
print(np.around(huz_CFC,
         3))

In [ ]:
np.diag(abs(huz_CFC))- np.diag(abs(mu_CFC))

In [ ]:
np.linalg.norm(np.diag(huz_CFC)- np.diag(mu_CFC))